Analiza i Bazy Danych
===============

Wojciech Żyła, piątek 10:45
---------------

### 1. Ile kategorii filmów mamy w wypożyczalni?

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

categories_count = pd.read_sql('SELECT COUNT(name) from category',con=connection)

print(f'Number of categories\n{categories_count}')

# In the database there are 16 categories of films.

Number of categories
   count
0     16


### 2. Wyświetl listę kategorii w kolejności alfabetycznej.

In [2]:
cat_sorted = pd.read_sql('SELECT name from category ORDER BY name',con=connection)
print(cat_sorted)

           name
0        Action
1     Animation
2      Children
3      Classics
4        Comedy
5   Documentary
6         Drama
7        Family
8       Foreign
9         Games
10       Horror
11        Music
12          New
13       Sci-Fi
14       Sports
15       Travel


### 3. Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [3]:

oldest = pd.read_sql('SELECT title, release_year from film ORDER BY release_year ASC LIMIT 1',con=connection)
youngest = pd.read_sql('SELECT title, release_year from film ORDER BY release_year DESC LIMIT 1',con=connection)
print(f"Najstarszy film\n{oldest}")
print("\n\n")
print(f"Najmłodszy film\n{youngest}")
 
# All films were produced in the same year.
# For this reason, the database takes the same movie as the oldest and youngest.

Najstarszy film
             title  release_year
0  Chamber Italian          2006



Najmłodszy film
             title  release_year
0  Chamber Italian          2006


### 4. Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [4]:
first_between = pd.read_sql("SELECT count(rental_id) FROM rental WHERE rental_date BETWEEN '2005-07-01' AND '2005-08-01'",con=connection)
print(first_between)

# Between 2005-07-01 and 2005-08-01 there were 6709 rentals.

   count
0   6709


### 5. Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [5]:
second_between = pd.read_sql("SELECT count(rental_id) FROM rental WHERE rental_date BETWEEN '2010-01-01' AND '2011-02-01'",con=connection)
print(second_between)

# Between 2010-01-01 and 2011-02-01 there were 0 rentals.

   count
0      0


### 6. Znajdź największą płatność wypożyczenia.

In [6]:
max_payment = pd.read_sql("SELECT MAX(amount) as MaxPayment FROM payment",con=connection)
print(max_payment)

# The biggest payment for rental was 11.99

   maxpayment
0       11.99


### 7. Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [7]:
clients = pd.read_sql("SELECT cu.first_name, co.country,  c.city \
                       FROM customer AS cu\
                       INNER JOIN\
                       address as a\
                       ON cu.address_id = a.address_id\
                       INNER JOIN\
                       city AS c\
                       ON a.city_id = c.city_id\
                       INNER JOIN\
                       country AS co\
                       ON c.country_id = co.country_id\
                       WHERE\
                       co.country IN ('Poland', 'Nigeria', 'Bangladesh')", con=connection)
print(clients)

# All of the clients regardless of the country they live in have english names.

   first_name     country             city
0       Sonia     Nigeria       Benin City
1       Brian      Poland        Bydgoszcz
2      Sidney      Poland      Czestochowa
3      Rodney     Nigeria        Deba Habe
4     Stephen  Bangladesh            Dhaka
5       Velma     Nigeria     Effon-Alaiye
6     Marilyn     Nigeria              Ife
7       Elsie     Nigeria           Ikerre
8      Gladys     Nigeria           Ilorin
9       Frank  Bangladesh         Jamalpur
10   Marjorie      Poland  Jastrzebie-Zdrj
11      Carol     Nigeria           Kaduna
12       Leah      Poland           Kalisz
13      Ruben      Poland           Lublin
14       Olga     Nigeria        Ogbomosho
15     Bertha     Nigeria             Ondo
16     Tracey     Nigeria              Owo
17         Jo     Nigeria              Oyo
18    Johnnie      Poland            Plock
19    Wallace     Nigeria           Sokoto
20   Michelle  Bangladesh          Tangail
21    Russell      Poland            Tychy
22     Jimm

### 8. Gdzie mieszkają członkowie personelu?

In [8]:
staff = pd.read_sql("SELECT s.first_name, co.country, c.city, a.address, a.address2\
                       FROM staff AS s\
                       INNER JOIN\
                       address as a\
                       ON s.address_id = a.address_id\
                       INNER JOIN\
                       city AS c\
                       ON a.city_id = c.city_id\
                       INNER JOIN\
                       country AS co\
                       ON c.country_id = co.country_id", con=connection)
print(staff)

  first_name    country        city               address address2
0       Mike     Canada  Lethbridge     23 Workhaven Lane     None
1        Jon  Australia   Woodridge  1411 Lillydale Drive     None


### 9. Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [9]:
staff_arg_sp = pd.read_sql("SELECT count(s.staff_id)\
                           FROM staff AS s\
                           INNER JOIN\
                           address as a\
                           ON s.address_id = a.address_id\
                           INNER JOIN\
                           city AS c\
                           ON a.city_id = c.city_id\
                           INNER JOIN\
                           country AS co\
                           ON c.country_id = co.country_id\
                           WHERE co.country IN ('Spain', 'Argentina')", con=connection)
print(staff_arg_sp)

# There are no employees living in Argentina or Spain. From previous exercise 
# we could have suspected that there are only two members of the staff who live in
# Canada and Australia.

   count
0      0


### 10. Jakie kategorie filmów zostały wypożyczone przez klientów?

In [10]:
cat_clients = pd.read_sql("SELECT DISTINCT c.name\
                           FROM rental AS r\
                           INNER JOIN\
                           inventory AS i\
                           ON r.inventory_id = i.inventory_id\
                           INNER JOIN\
                           film AS f\
                           ON i.film_id = f.film_id\
                           INNER JOIN\
                           film_category AS fc\
                           ON f.film_id = fc.film_id\
                           INNER JOIN\
                           category AS c\
                           ON fc.category_id = c.category_id", con=connection)

print(cat_clients)

           name
0        Horror
1        Comedy
2         Drama
3        Sci-Fi
4       Foreign
5      Classics
6         Games
7        Travel
8           New
9         Music
10       Action
11  Documentary
12       Sports
13    Animation
14     Children
15       Family


### 11. Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [11]:
distinct_countries = pd.read_sql("SELECT country FROM country", con=connection)

# Names of the countries
for i, row in distinct_countries.iterrows():
    print(row["country"])

Afghanistan
Algeria
American Samoa
Angola
Anguilla
Argentina
Armenia
Australia
Austria
Azerbaijan
Bahrain
Bangladesh
Belarus
Bolivia
Brazil
Brunei
Bulgaria
Cambodia
Cameroon
Canada
Chad
Chile
China
Colombia
Congo, The Democratic Republic of the
Czech Republic
Dominican Republic
Ecuador
Egypt
Estonia
Ethiopia
Faroe Islands
Finland
France
French Guiana
French Polynesia
Gambia
Germany
Greece
Greenland
Holy See (Vatican City State)
Hong Kong
Hungary
India
Indonesia
Iran
Iraq
Israel
Italy
Japan
Kazakstan
Kenya
Kuwait
Latvia
Liechtenstein
Lithuania
Madagascar
Malawi
Malaysia
Mexico
Moldova
Morocco
Mozambique
Myanmar
Nauru
Nepal
Netherlands
New Zealand
Nigeria
North Korea
Oman
Pakistan
Paraguay
Peru
Philippines
Poland
Puerto Rico
Romania
Runion
Russian Federation
Saint Vincent and the Grenadines
Saudi Arabia
Senegal
Slovakia
South Africa
South Korea
Spain
Sri Lanka
Sudan
Sweden
Switzerland
Taiwan
Tanzania
Thailand
Tonga
Tunisia
Turkey
Turkmenistan
Tuvalu
Ukraine
United Arab Emirates
United Ki

In [12]:
cat_in_usa = pd.read_sql("SELECT DISTINCT c.name\
                           FROM rental AS r\
                           INNER JOIN\
                           inventory AS i\
                           ON r.inventory_id = i.inventory_id\
                           INNER JOIN\
                           film AS f\
                           ON i.film_id = f.film_id\
                           INNER JOIN\
                           film_category AS fc\
                           ON f.film_id = fc.film_id\
                           INNER JOIN\
                           category AS c\
                           ON fc.category_id = c.category_id\
                           INNER JOIN\
                           store AS st\
                           ON i.store_id = st.store_id\
                           INNER JOIN\
                           address as adr\
                           ON st.address_id = adr.address_id\
                           INNER JOIN\
                           city AS cit\
                           ON adr.city_id = cit.city_id\
                           INNER JOIN\
                           country AS cou\
                           ON cit.country_id = cou.country_id\
                           WHERE cou.country = 'United States'", con=connection)
print(cat_in_usa)

# There are no rentals in USA.

Empty DataFrame
Columns: [name]
Index: []


In [13]:
rent_countries = pd.read_sql("SELECT DISTINCT c.country\
                              FROM rental AS r\
                              INNER JOIN\
                              inventory AS i\
                              ON r.inventory_id = i.inventory_id\
                              INNER JOIN\
                               store AS s\
                               ON i.store_id = s.store_id\
                               INNER JOIN\
                               address as a\
                               ON s.address_id = a.address_id\
                               INNER JOIN\
                               city AS cit\
                               ON a.city_id = cit.city_id\
                               INNER JOIN\
                               country AS c\
                               ON cit.country_id = c.country_id", con=connection)
print(rent_countries)

# The only two countries where films were rented are Australia and Canada.

     country
0  Australia
1     Canada


### 12. Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [14]:
films_actors = pd.read_sql("SELECT title, a.first_name, a.last_name\
                            FROM film AS f\
                            INNER JOIN\
                            film_actor AS fa\
                            ON f.film_id = fa.film_id\
                            INNER JOIN\
                            actor AS a\
                            ON fa.actor_id = a.actor_id\
                            WHERE (a.first_name = 'Olympia' AND a.last_name = 'Pfeiffer')\
                            OR (a.first_name = 'Julia' AND a.last_name = 'Zellweger')\
                            OR (a.first_name = 'Ellen' AND a.last_name = 'Presley')", con=connection)
# Print titles
for i, row in films_actors.iterrows():
    print(f"{row['title']} | {row['first_name']} | {row['last_name']}")

Bilko Anonymous | Ellen | Presley
Caribbean Liberty | Ellen | Presley
Casper Dragonfly | Ellen | Presley
Empire Malkovich | Ellen | Presley
Floats Garden | Ellen | Presley
Frogmen Breaking | Ellen | Presley
Homeward Cider | Ellen | Presley
Hyde Doctor | Ellen | Presley
Image Princess | Ellen | Presley
Jacket Frisco | Ellen | Presley
Microcosmos Paradise | Ellen | Presley
Network Peak | Ellen | Presley
Oscar Gold | Ellen | Presley
Pickup Driving | Ellen | Presley
Pinocchio Simon | Ellen | Presley
Private Drop | Ellen | Presley
Roots Remember | Ellen | Presley
Scarface Bang | Ellen | Presley
Secretary Rouge | Ellen | Presley
Spy Mile | Ellen | Presley
Streetcar Intentions | Ellen | Presley
Tadpole Park | Ellen | Presley
Treasure Command | Ellen | Presley
Turn Star | Ellen | Presley
Women Dorado | Ellen | Presley
Badman Dawn | Olympia | Pfeiffer
Chitty Lock | Olympia | Pfeiffer
Color Philadelphia | Olympia | Pfeiffer
Contact Anonymous | Olympia | Pfeiffer
Deep Crusade | Olympia | Pfeiffer